# Publications markdown generator for academicpages

Takes a TSV of publications with metadata and converts them for use with [academicpages.github.io](academicpages.github.io). This is an interactive Jupyter notebook ([see more info here](http://jupyter-notebook-beginner-guide.readthedocs.io/en/latest/what_is_jupyter.html)). The core python code is also in `publications.py`. Run either from the `markdown_generator` folder after replacing `publications.tsv` with one containing your data.

TODO: Make this work with BibTex and other databases of citations, rather than Stuart's non-standard TSV format and citation style.


## Data format

The TSV needs to have the following columns: pub_date, title, venue, excerpt, citation, site_url, and paper_url, with a header at the top. 

- `excerpt` and `paper_url` can be blank, but the others must have values. 
- `pub_date` must be formatted as YYYY-MM-DD.
- `url_slug` will be the descriptive part of the .md file and the permalink URL for the page about the paper. The .md file will be `YYYY-MM-DD-[url_slug].md` and the permalink will be `https://[yourdomain]/publications/YYYY-MM-DD-[url_slug]`

This is how the raw file looks (it doesn't look pretty, use a spreadsheet or other program to edit and create).

In [ ]:
!cat publications.tsv

pub_date	title	venue	excerpt	citation	url_slug	paper_url
2009-10-01	Paper Title Number 1	Journal 1	This paper is about the number 1. The number 2 is left for future work.	Your Name, You. (2009). "Paper Title Number 1." <i>Journal 1</i>. 1(1).	paper-title-number-1	http://academicpages.github.io/files/paper1.pdf
2010-10-01	Paper Title Number 2	Journal 1	This paper is about the number 2. The number 3 is left for future work.	Your Name, You. (2010). "Paper Title Number 2." <i>Journal 1</i>. 1(2).	paper-title-number-2	http://academicpages.github.io/files/paper2.pdf
2015-10-01	Paper Title Number 3	Journal 1	This paper is about the number 3. The number 4 is left for future work.	Your Name, You. (2015). "Paper Title Number 3." <i>Journal 1</i>. 1(3).	paper-title-number-3	http://academicpages.github.io/files/paper3.pdf

## Import pandas

We are using the very handy pandas library for dataframes.

In [ ]:
import pandas as pd

## Import TSV

Pandas makes this easy with the read_csv function. We are using a TSV, so we specify the separator as a tab, or `\t`.

I found it important to put this data in a tab-separated values format, because there are a lot of commas in this kind of data and comma-separated values can get messed up. However, you can modify the import statement, as pandas also has read_excel(), read_json(), and others.

In [ ]:
publications = pd.read_csv("publications.tsv", sep="\t", header=0)
publications


,pub_date,title,venue,excerpt,citation,url_slug,paper_url
0,2009-10-01,Paper Title Number 1,Journal 1,This paper is about the number 1. The number 2...,"Your Name, You. (2009). ""Paper Title Number 1....",paper-title-number-1,http://academicpages.github.io/files/paper1.pdf
1,2010-10-01,Paper Title Number 2,Journal 1,This paper is about the number 2. The number 3...,"Your Name, You. (2010). ""Paper Title Number 2....",paper-title-number-2,http://academicpages.github.io/files/paper2.pdf
2,2015-10-01,Paper Title Number 3,Journal 1,This paper is about the number 3. The number 4...,"Your Name, You. (2015). ""Paper Title Number 3....",paper-title-number-3,http://academicpages.github.io/files/paper3.pdf


## Read `papers.bib`

We are reading a `.bib` containing all publications

In [ ]:
!pip install pybtex

In [ ]:
import os
import pandas as pd
import pathlib
from collections import defaultdict
from pybtex.database.input import bibtex

In [ ]:
parser = bibtex.Parser()
bibdata = parser.parse_file("files/papers.bib")
fields = ['title', 'journal', 'volume', 'number', 'pages', 'year', 'publisher']

#loop through the individual references
cite = defaultdict(list)
for bib_id, item in bibdata.entries.items():
  cite['bib_id'].append(bib_id)
  b = item.fields
  for field in fields:
    cite[field].append(b[field]) if field in b.keys() else cite[field].append('None')
  authors = [str(person).title() for person in bibdata.entries[bib_id].persons['author']]
  cite['authors'].append(authors)
  cite['bib_id'].append(bib_id)
publications = pd.DataFrame.from_dict(cite)

In [ ]:
bibdata

In [ ]:
# saving as tsv file
publications.to_csv('markdown_generator/publications.tsv', sep="\t")

## Escape special characters

YAML is very picky about how it takes a valid string, so we are replacing single and double quotes (and ampersands) with their HTML encoded equivilents. This makes them look not so readable in raw format, but they are parsed and rendered nicely.

In [ ]:
html_escape_table = {
    "&": "&amp;",
    '"': "&quot;",
    "'": "&apos;"
    }

def html_escape(text):
    """Produce entities within text."""
    return "".join(html_escape_table.get(c,c) for c in text)

## Creating the markdown files

This is where the heavy lifting is done. This loops through all the rows in the TSV dataframe, then starts to concatentate a big string (```md```) that contains the markdown for each type. It does the YAML metadata first, then does the description for the individual page.

In [27]:
for row, item in publications.iterrows():
  md_filename = str(f"{item.year}-01-01") + "-" + item.bib_id + ".md"
  html_filename = str(f"{item.year}-01-01") + "-" + item.bib_id
  year = item.year

  ## YAML variables
  md = "---\ntitle: \""   + item.title + '"\n'
  md += """collection: publications"""
  md += """\npermalink: /publication/""" + html_filename

  # if len(str(item.excerpt)) > 5:
  #   md += "\nexcerpt: '" + html_escape(item.excerpt) + "'"
  
  venue = str(item.journal) if item.journal != 'None' else str(item.publisher)

  md += "\ndate: " + str(f"{item.year}-01-01") 
  md += "\nvenue: '" + html_escape(venue) + "'"

  # if len(str(item.paper_url)) > 5:
  #   md += "\npaperurl: '" + item.paper_url + "'"

  authors = ", ".join(item.authors)
  citation = f"{authors} ({item.year}), {item.title}"
  md += "\ncitation: '" + html_escape(citation) + "'"
  md += "\n---"

  ## Markdown description for individual page
  # if len(str(item.excerpt)) > 5:
  #   md += "\n" + html_escape(item.excerpt) + "\n"

  # if len(str(item.paper_url)) > 5:
  #   md += "\n[Download paper here](" + item.paper_url + ")\n" 
      
  md += "\nRecommended citation: " + citation

  pathlib.Path('_publications/').mkdir(parents=True, exist_ok=True)

  md_filename = os.path.basename(md_filename)
  with open("_publications/" + md_filename, 'w') as f:
      f.write(md)

These files are in the publications directory, one directory below where we're working from.

In [26]:
!ls _publications

ls: cannot access '_publications': No such file or directory


In [ ]:
!cat _publications/2010-ezeani2010heuristic.md

---
title: "Heuristic Approaches to Solving MultiObjective Scheduling Optimisation Problems"
collection: publications
permalink: /publication/2010-ezeani2010heuristic
date: 2010
venue: 'Electroscope Journal'
citation: 'Ezeani, Mi, Agbasi, Kc, Osita, Eg, Okonkwo, Ro, Nnanwa, Cp (2010), Heuristic Approaches to Solving MultiObjective Scheduling Optimisation Problems'
---
Recommended citation: Ezeani, Mi, Agbasi, Kc, Osita, Eg, Okonkwo, Ro, Nnanwa, Cp (2010), Heuristic Approaches to Solving MultiObjective Scheduling Optimisation Problems